# Loan default prediction challenge

# Data description

Kowope Mart is a Nigerian-based retail company with a vision to provide quality goods, education and automobile services to its customers at affordable price and reduce if not eradicate charges on card payments and increase customer satisfaction with credit rewards that can be used within the Mall. To achieve this, the company has partnered with DSBank on co-branded credit card with additional functionality such that customers can request for loan, pay for goods even with zero-balance and then pay back within an agreed period of time. This innovative strategy has increased sales for the company. However, there has been recent cases of credit defaults and Kowope Mart will like to have a system that profiles customers who are worthy of the card with minimum if not zero risk of defaulting.

The data set can be accessed using the link below: https://zindi.africa/hackathons/dsn-ai-bootcamp-qualification-hackathon
        
The basis of the project is to predict customers who are likely to default or not.

default_status: defaulted or not. (yes:1, no: 0)

In [1]:
#Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

import os, sys
import warnings
warnings.filterwarnings('ignore')
import pandas.util.testing as tm

In [3]:
train = pd.read_csv('/content/drive/My Drive/Colab/loan_train.csv')

test = pd.read_csv('/content/drive/My Drive/Colab/loan_test.csv')

In [4]:
train.shape, test.shape

((56000, 52), (24000, 51))

In [5]:
data = train.copy()

In [7]:
data.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139.0,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427.0,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457.0,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375.0,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427.0,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [8]:
def func(data):
    d =[]
    for m in data:
        if m =='yes':
            d.append(1)
        else:
            d.append(0)
    return d

data['default_status'] = func(data['default_status'])

In [9]:
dict_formfield47 = {'charge': 1,
         'lending': 0}


data['form_field47'].replace(dict_formfield47, inplace = True)
test['form_field47'].replace(dict_formfield47, inplace = True)

In [11]:
data.fillna(-999, inplace = True)
test.fillna(-999, inplace = True)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     56000 non-null  float64
 2   form_field2     56000 non-null  float64
 3   form_field3     56000 non-null  float64
 4   form_field4     56000 non-null  float64
 5   form_field5     56000 non-null  float64
 6   form_field6     56000 non-null  float64
 7   form_field7     56000 non-null  float64
 8   form_field8     56000 non-null  float64
 9   form_field9     56000 non-null  float64
 10  form_field10    56000 non-null  float64
 11  form_field11    56000 non-null  float64
 12  form_field12    56000 non-null  float64
 13  form_field13    56000 non-null  float64
 14  form_field14    56000 non-null  float64
 15  form_field15    56000 non-null  float64
 16  form_field16    56000 non-null  float64
 17  form_field17    56000 non-null 

In [13]:
df_features = data.select_dtypes(exclude = object).columns.drop(['default_status'])

In [14]:
x = data[df_features]
y = data['default_status']

In [15]:
!pip install catboost

     |████████████████████████████████| 66.1MB 60kB/s 


In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold

In [17]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y, 
                                                    test_size = 0.2,
                                                    random_state = 42)

In [19]:
model1 = GradientBoostingClassifier()
model1.fit(x_train,y_train)# model learning

print('Training score is :', model1.score(x_train, y_train))
pred1 = model1.predict_proba(x_test)[:,1]

Training score is : 0.8170089285714286


In [20]:
model2 = XGBClassifier()
model2.fit(x_train,y_train)# model learning

print('Training score is :', model2.score(x_train, y_train))
pred2 = model2.predict_proba(x_test)[:,1]

Training score is : 0.8154241071428572


In [21]:
model3 = lgb.LGBMClassifier()
model3.fit(x_train, y_train)
print('Training score is :', model3.score(x_train, y_train))
pred3 = model3.predict_proba(x_test)[:,1]

Training score is : 0.8400892857142858


In [22]:
model4 = CatBoostClassifier()
model4.fit(x_train, y_train)
print('Training score is :', model4.score(x_train, y_train))
pred = model4.predict_proba(x_test)[:,1]

Learning rate set to 0.052243
0:	learn: 0.6589894	total: 87.4ms	remaining: 1m 27s
1:	learn: 0.6305818	total: 122ms	remaining: 1m
2:	learn: 0.6052075	total: 159ms	remaining: 52.8s
3:	learn: 0.5856726	total: 194ms	remaining: 48.3s
4:	learn: 0.5671607	total: 232ms	remaining: 46.1s
5:	learn: 0.5505941	total: 268ms	remaining: 44.4s
6:	learn: 0.5361869	total: 309ms	remaining: 43.8s
7:	learn: 0.5240854	total: 344ms	remaining: 42.7s
8:	learn: 0.5135773	total: 380ms	remaining: 41.8s
9:	learn: 0.5050148	total: 413ms	remaining: 40.8s
10:	learn: 0.4962478	total: 454ms	remaining: 40.8s
11:	learn: 0.4884398	total: 493ms	remaining: 40.6s
12:	learn: 0.4816486	total: 528ms	remaining: 40.1s
13:	learn: 0.4758698	total: 563ms	remaining: 39.7s
14:	learn: 0.4709219	total: 599ms	remaining: 39.3s
15:	learn: 0.4658448	total: 635ms	remaining: 39.1s
16:	learn: 0.4614520	total: 674ms	remaining: 39s
17:	learn: 0.4571995	total: 710ms	remaining: 38.7s
18:	learn: 0.4534078	total: 745ms	remaining: 38.4s
19:	learn: 0.4

In [23]:
from mlxtend.classifier import StackingCVClassifier
RANDOM_SEED = 42

stack = StackingCVClassifier(classifiers=(model1, model2, model3),
                            meta_classifier=model4, cv=12,
                            use_features_in_secondary=True,
                            store_train_meta_features=True,
                            shuffle=False,
                            use_probas=True
                            )

stack.fit(x_train, y_train)
pred = stack.predict(x_test)


KeyError: ignored

In [ ]:
# # Specify number of folds
# n_skf = 15
# kf = StratifiedKFold(n_skf)


# seed = 2020

# ## Manually tweak the hyperparameter, learning rate and n_estimators especially
# params = {
#     'n_estimators': 5000,
#     'learning_rate': 0.01,
#     'objective': 'CrossEntropy',
#       'eval_metric':'AUC',
#     'random_seed': seed,
#     'early_stopping_rounds': 200,
#     'use_best_model': True,
#     'max_depth': 8,
#     # 'subsample' : 0.8,
#     # 'bagging_temperature' : 0.8,
#     # 'random_strength' :10,
#     # 'one_hot_max_size': 15,
}

In [ ]:
# score_list = []
# score = 0
# test_oofs = []

# for i, (tr_idx, vr_idx) in enumerate(kf.split(x, y)):
    
#     xtrain, ytrain = x.loc[tr_idx, df_features], y.loc[tr_idx]
#     xval, yval = x.loc[vr_idx, df_features], y.loc[vr_idx]

#     model = CatBoostClassifier(**params)
#     model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
#     p = model.predict_proba(xval)[:, 1]
#     sc = metric(yval, p)
#     score_list.append(sc)
#     score += sc/n_skf
    
#     pred = model.predict_proba(test[df_features])[:, 1]
#     test_oofs.append(pred)

#     print('Fold {} : {}'.format(i, sc))

# print()
# print()
# print('Avg log : ', score)

0:	test: 0.7901639	best: 0.7901639 (0)	total: 61ms	remaining: 6m 5s
100:	test: 0.8153087	best: 0.8153195 (97)	total: 6.01s	remaining: 5m 51s
200:	test: 0.8180593	best: 0.8180621 (199)	total: 11.9s	remaining: 5m 41s
300:	test: 0.8198020	best: 0.8198020 (300)	total: 17.7s	remaining: 5m 35s
400:	test: 0.8212365	best: 0.8212493 (398)	total: 23.4s	remaining: 5m 27s
500:	test: 0.8224542	best: 0.8224569 (499)	total: 29s	remaining: 5m 18s
600:	test: 0.8232819	best: 0.8232819 (600)	total: 34.7s	remaining: 5m 11s
700:	test: 0.8239828	best: 0.8239867 (695)	total: 40.4s	remaining: 5m 5s
800:	test: 0.8249253	best: 0.8249253 (800)	total: 46s	remaining: 4m 58s
900:	test: 0.8255921	best: 0.8255921 (900)	total: 51.7s	remaining: 4m 52s
1000:	test: 0.8261000	best: 0.8261000 (1000)	total: 57.3s	remaining: 4m 46s
1100:	test: 0.8265749	best: 0.8265749 (1100)	total: 1m 3s	remaining: 4m 41s
1200:	test: 0.8270564	best: 0.8270619 (1199)	total: 1m 8s	remaining: 4m 35s
1300:	test: 0.8273957	best: 0.8273957 (1300)

In [ ]:
#f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'15 fold CV, score: 0.8416217882140743'

In [ ]:
#oof_prediction = pd.DataFrame(test_oofs).T

In [ ]:
#oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [ ]:
#oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_11,fold_12,fold_13,fold_14,fold_15
0,0.314193,0.283159,0.310480,0.324163,0.316508,0.288189,0.278254,0.321448,0.302994,0.314130,0.307477,0.291216,0.288866,0.307376,0.281149
1,0.463516,0.415970,0.429711,0.452438,0.387679,0.439844,0.438632,0.465311,0.369241,0.391125,0.351248,0.434518,0.401933,0.440222,0.378573
2,0.350431,0.359852,0.363995,0.366127,0.373124,0.384570,0.381708,0.370590,0.382899,0.403235,0.383760,0.363201,0.348107,0.355378,0.364854
3,0.729472,0.740849,0.760896,0.748626,0.740182,0.757619,0.739363,0.746837,0.739013,0.736693,0.757996,0.749665,0.730141,0.747193,0.733266
4,0.164313,0.176318,0.185562,0.182584,0.142928,0.171719,0.171412,0.169724,0.153198,0.150926,0.149851,0.169031,0.183441,0.160850,0.159744


In [ ]:
#sub = pd.read_csv('/content/drive/My Drive/Colab/SampleSubmission.csv')

In [ ]:
#sub['default_status'] = np.mean(test_oofs, axis = 0) 

In [ ]:
#data = sub.to_csv('cat_classifier7.csv', index= False)

In [ ]:
#from google.colab import files

In [ ]:
#files.download('cat_classifier7.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>